In [2]:
import random

## 9.0 相关概念

- **顺序统计量**
  - 第 $i$ 个顺序统计量是指集合中第 $i$ 小的元素
  - $i=1$ 时，为**最小值**
  - $i=n$ 时，为**最大值**
  - 当 $n$ 为奇数时， $i = (n+1)/2$ 为**中位数**
  - 当 $n$ 为奇数时，存在两个中位数，分别位为 $ i = n/2$ 和 $i = n/2 + 1$ 处
  - 不考虑 $n$ 的奇偶性，则中位数总是出现在 $i=\lfloor (n+1)/2 \rfloor$(**下中位数**) 和 $\lceil (n+2)/2 \rceil$ 处(**上中位数**)

- **选择问题**  
  - 寻找第 $i$ 个顺序统计量，可以抽像为选择问题

> 输入：一个包含 $n$ 个（互异的）数的集合 $A$ 和一个整数 $i$, $1 \le i \le n$  
输出： 元素 $x \in i$, 且$A$中恰好有 $i-1$ 个元素小于它 

## 9.1 最大值和最小值

### 找出最小值

- 可通过 $n-1$ 次比较得出

In [3]:
def minimum(A):
  mini = A[0]
  for i in range(1, len(A)):
    if A[i] < mini:
      mini = A[i]
  return mini

In [4]:
A = random.sample(range(100), 10)
print("A = {}".format(A))
print("{} is the minimun item in A".format(minimum(A)))

A = [26, 62, 75, 84, 27, 87, 54, 74, 37, 63]
26 is the minimun item in A


### 找出最大值

- 与最小值一样，也可通过 $n-1$ 次比较得到

In [5]:
def maxmum(A):
  maxm = A[0]
  for i in range(1, len(A)):
    if A[i] > maxm:
      maxm = A[i]
  return maxm

In [6]:
A = random.sample(range(100), 10)
print("A = {}".format(A))
print("{} is the maxmun item in A".format(maxmum(A)))

A = [49, 47, 68, 73, 38, 95, 79, 88, 29, 63]
95 is the maxmun item in A


### 同时找出最大值和最小值

- 通过成对的处理输入，最多只需要 $3\lfloor n/2 \rfloor$ 次比较即可同时找出最大值和最小值
  - 先比较两个输元素，然后将较小的值与当前的最小值进行比较，将较大的值与当前的最大值进行比较

In [7]:
def mini_maxmum(A):
  n = len(A)
  if n % 2 == 0:
    mini, maxm = A[0], A[1]
    start_index = 2
  else:
    mini = maxm = A[0]
    start_index = 1
  for i in range(start_index, len(A), 2):
    if A[i] > A[i+1]:
      tmp_mini, tmp_maxm = A[i+1], A[i]
    else:
      tmp_mini, tmp_maxm = A[i], A[i+1]
    if tmp_mini < mini:
      mini = tmp_mini
    if tmp_maxm > maxm:
      maxm = tmp_maxm
  return mini, maxm
  

In [8]:
A = random.sample(range(100), random.randint(10, 20))
print("A = {}".format(A))
mini, maxm = mini_maxmum(A)
print("{} is the minimun item in A, {} is the maxmun item in A.".format(mini, maxm))

A = [65, 62, 66, 60, 55, 96, 0, 61, 51, 32, 46, 79, 8, 33, 21]
0 is the minimun item in A, 96 is the maxmun item in A.


### 练习

#### 9.1-1 
经过 $n + \lceil lgn \rceil - 2$ 次比较找到 $n$ 个元素中第二小的元素

$${n \over 2} + {n \over 2^2} + \cdots + {1}$$
$${1-2^{lgn} \over 1-2 }= n - 1$$ 

## 9-2 期望为线性时间的选择算法

### 代码实现

- 选择排序的分治算法 RANDOMIZED-SELECT 是以快速排序算法为基础的
- 快速排序算法会递归的处理 partition 后两边的数组，而 RANDOMIZED-SELECT 只用处理一边的数组

In [9]:
def partition(A, p, r):
  """以最后一个元素为主元对数组 A 进行分组"""
  q = p - 1
  for j in range(p, r):
    if A[j] < A[r]:
      q += 1
      A[j], A[q] = A[q], A[j]
  A[q+1], A[r] = A[r], A[q+1]
  return q + 1

In [17]:
def randomized_partition(A, p, r):
  """随机选择主元对数组 A 进行划分"""
  i = random.randint(p, r)
  A[i], A[r] = A[r], A[i]
  return partition(A, p, r)

In [60]:
def randomized_select(A, p, r, i):
  """选择数组 A 中在 [p, r] 中的第 k 个顺序统计量"""
  if p == r:
    return A[p]
  q = randomized_partition(A, p, r)
  k = q - p + 1
  if k == i:
    return A[q]
  elif k > i:
    return randomized_select(A, p, q-1, i)
  else:
    return randomized_select(A, q+1, r, i-k)

In [67]:
A = [random.randint(1, 15) for i in range(10)]
print("A = {}".format(A))
i = random.randint(1, 10)
print("A 中的第 {} 个顺序统计量为 {}".format(i, randomized_select(A, 0, len(A)-1, i)))
print("排序后，第 {} 个值为 {}".format(i, sorted(A)[i-1]))

A = [15, 3, 5, 4, 3, 7, 5, 15, 6, 11]
A 中的第 5 个顺序统计量为 5
排序后，第 5 个值为 5


### 期望运行时间的上界

- 定时指标器随机变量：
$$X_k = I\{子数组 \ A[p \cdots q]\} \ 正好包含 k 个元素$$  
  - 假设元素是互异的，有 $$ E[X_k] = 1/n$$
- 根据指示器随机变量的定义，可得下述递归式:  $$T(n) \le \sum_{k=1}^{n}{X_k \times (T(max(k-1, n-k)) + O(n))}$$
- 根据期望的相关性质和代入法，可证得：$$E(T(n)) = O (N)$$

### 练习题

$$ p > q - 1 \\
  q - p  \le 0 \\
  k \le 1 \\
$$

$$
q + 1 > r \\
q - p + 1 > r - p\\
k > r - p
$$

## 9.3 最坏情况下为线性时间的选择算法

### SELECT 算法描述  
  1. 将输入数组的 $n$ 个元素划分为 $\lfloor n/5 \rfloor$ 组，每组 $5$ 个元素，则至多还有一组由剩下的 $n \ mod \ 5 $ 个元素组成
  2. 确定 $\lceil n/5 \rceil$ 组每一组的中位数, 可用插入排序来实现
  3. 对于第 2 步中确认的每组的中位数，递归的调用 SELECT 以找出其下中位数 $x$，
  4. 以 $x$ 为主元，调用 PARTITION，对数组进行分组，设 $x$ 为第 $k$ 小的元素，则有 $n-k$ 个元素在划分的高区
  5. 如果 $i=k$，则返回 $x$, 如果$i < k$, 则在低区递归的调用 SELECT 来找出第 $i$ 小的元素，如果 $i>k$, 则在高区递归的查找第 $i-k$ 小的元素

### 代码实现

In [120]:
def find_median(A, p, r):
  """借助插入排序找出数组 A 指定区域的下中位数"""
  j = p + 1
  for j in range(p+1, r+1):
    i = j - 1
    key = A[j]
    while i >=p and A[i] > key:
      A[i+1] = A[i]
      i -= 1
    A[i+1] = key
  return A[(p+r)//2]

In [123]:
def select(A, p, r, i):
  if r - p <= 5:
    find_median(A, p, r)
    return A[p + i - 1]
  j = p
  medians = list()
  while True:
    if j+5 <= r:
      medians.append(find_median(A, j, j+5))
      j += 5
    else:
      medians.append(find_median(A, j, r)) 
      break
  A[r] = select(medians, 0, len(medians)-1, (len(medians)- 1)//2)
  q = partition(A, p, r)
  k = q - p + 1
  if k == i:
    return A[q]
  elif k > i:
    return select(A, p, q-1, i)
  else:
    return select(A, q+1, r, i-k)

In [177]:
A = [random.randint(1, 10000) for i in range(random.randint(1, 1000))]
i = random.randint(1, len(A))
print("select 输出的结果: {}".format(select(A, 0, len(A)-1, i)))
print("先排序，再取第 i 个值的结果: {}".format(sorted(A)[i-1])) 

select 输出的结果: 1827
先排序，再取第 i 个值的结果: 1827


### 时间复杂度分析

- 当输入规模为 $n$ 时，设其最坏情况下的运行时间为 $T(n)$
- 第 3 步的运行时间为 $T(\lceil n/5 \rceil)$  
  - 进一步分析，只要第 5 步中递归调用的规模严格的小于 $4n/5$, 即可保证整个算法为线性算法
- 第 4 步划分后，大于 $x$ 的个数至少为: $$3(\lceil{{1 \over 2} \lceil {n \over 5} \rceil} \rceil - 2) \ge {3n \over 10} - 6 $$  
  - 分析过程图示：
    - <img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191226083609.png" width = 300>
  - 则第 5 步中，SELECT 递归调用时，最多作用于 $7n\ /\ 10 + 6$ 个元素
- 经过上述分析，可得： $$T(n) \le T(\lceil n/5 \rceil) + T(7n/10) + 6 + O(n)$$
